<a href="https://colab.research.google.com/github/geoalinho/cascata-do-salso/blob/main/Cascata%20do%20Salso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import requests

from pathlib import Path

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-alansvolta')

In [ ]:
class AreaOfInterest:
  def __init__():
    pass

salso_cascade_point = ee.Geometry.Point([-53.448807548002016, -30.569416210967997]  )
salso_cascade_area = salso_cascade_point.buffer(1000)
salso_cascade_area = salso_cascade_area.bounds()

In [ ]:
class DigitalElevationModel:
  def __init__(self, dem_id, *, band=None, bounds=None):
    self.id = dem_id

    if band:
      self.image = ee.Image(self.id).select(band)
    else:
      self.image = ee.Image(self.id)

    if bounds:
      self.image = self.image.clip(bounds)

  def hillshade(self, azimuth, angle_altitude):
    return ee.Terrain.hillshade(self.image, azimuth, angle_altitude)

  def aspect(self):
    return ee.Terrain.aspect(self.image)

In [ ]:
class Palettes:
  @staticmethod
  def green_yellow_red():
    return ['green', 'yellowgreen', 'yellow', 'goldenrod', 'orange', 'red']

  @staticmethod
  def blues():
    return ['white', 'lightblue', 'blue']

  @staticmethod
  def rainbow():
    return ['darkblue', 'blue', 'lightblue', 'green', 'yellow', 'orange', 'red', 'white']

In [ ]:
class VisualizationParameters:
  @staticmethod
  def min_and_max(image, band, *, palette=None):
    if(palette == None):
      palette = Palettes.rainbow()

    min = ee.Number(image.select(band).reduceRegion(reducer=ee.Reducer.min(), geometry=salso_cascade_area, scale=10).get(band))
    max = ee.Number(image.select(band).reduceRegion(reducer=ee.Reducer.max(), geometry=salso_cascade_area, scale=10).get(band))
    print(f'min: {min.getInfo()}')
    print(f'max: {max.getInfo()}')
    return {'min': min,
            'max': max,
            'palette': palette}

  def cumulative_cut(image, band, *, palette=None, min=2, max=98):
    if(palette == None):
      palette = Palettes.rainbow()

    min = ee.Number(image.select(band).reduceRegion(reducer=ee.Reducer.percentile([min]), geometry=salso_cascade_area, scale=10).get(band))
    max = ee.Number(image.select(band).reduceRegion(reducer=ee.Reducer.percentile([max]), geometry=salso_cascade_area, scale=10).get(band))
    print(f'min: {min.getInfo()}')
    print(f'max: {max.getInfo()}')
    return {'min': min,
            'max': max,
            'palette': palette}

In [ ]:
# class FileManager:
#   def __init__(self):
#     self.colab_content_path = '/content/'

#   staticmethod
#   def clear_colab_content(self, *, extensions=None):
#     pass
#     # path = Path(self.colab_content_path)

#     # for file in path.iterdir():
#     #   if extensions is None:
#     #     try:
#     #         file.unlink()
#     #     except Exception:
#     #         pass
#     #   else:
#     #     file.is_file() and file.suffix[1:] in self.extensions:
#     #     try:
#     #       file.unlink()
#     #     except Exception:
#     #         pass

# class Downloader:
#   staticmethod
#   def ee_image_to_colab(image, filename, params):
#     url = image.getDownloadURL(params)
#     response = requests.get(url)

#     with open(filename, 'wb') as f:
#       f.write(response.content)

# class Uploader:
#   staticmethod
#   def desktop_vector_to_colab()

# class BoundaryManager:
#   def __init__(self):
#     self.extensions = {
#                       'cpg': False,
#                       'dbf': True,
#                       'prj': True,
#                       'shp': True,
#                       'shx': True,
#                       'qix': False,
#                       'qmd': False,
#                       }
#     self.colab_content = '/content/'
#     self.colab_boundary_path = None
#     self.ee_assets = 'projects/historic-vigor/assets/boundaries'
#     self.ee_boundary_id = None
#     self.boundary_crs = None

#   def clear_colab_content(self) -> None:
#     for file in os.listdir(self.colab_content):
#       ext = file.split('.')[-1]

#       if ext in self.extensions:
#         try:
#           os.remove(os.path.join(self.colab_content, file))
#         except:
#           pass

#     return

#   def clear_ee_assets(self) -> None:
#     assets = ee.data.listAssets(self.ee_assets)['assets']

#     for asset in assets:
#       ee.data.deleteAsset(asset['name'])

#     return

#   def create_ee_boundaries_folder(self):
#     try:
#       ee.data.createFolder(path=self.clear_ee_assets)
#     except:
#       pass

#     return

#   def upload_boundary(self) -> None:
#     self.clear_colab_content()

#     uploaded = files.upload()
#     boundary_path = None

#     for file in uploaded:
#       file_extension = file.split('.')[-1]

#       if file_extension == 'shp':
#         boundary_path = file

#       if not file_extension in self.extensions:
#         self.delete(file)

#     self.colab_boundary_path = boundary_path
#     self.set_boundary_crs()

#     return

#   def set_boundary_crs(self) -> None:
#     gdf = gpd.read_file(self.colab_boundary_path)
#     self.boundary_crs = str(gdf.crs)

#     return

#   def get_boundary_crs(self) -> None:
#     return self.boundary_crs

#   def to_ee_asset(self) -> None:
#     self.create_ee_boundaries_folder()
#     self.clear_ee_assets()

#     file_name = self.colab_boundary_path.split('.')[0]
#     asset_id = f'{self.ee_assets}/{file_name}'

#     gdf = gpd.read_file(self.colab_boundary_path)
#     gdf = gdf.to_crs('EPSG:4326')

#     geojson_data = gdf.__geo_interface__

#     fc = geemap.geojson_to_ee(geojson_data)

#     task = ee.batch.Export.table.toAsset(
#         collection=fc,
#         description='Importacao_de_contorno',
#         assetId=asset_id
#     )
#     task.start()

#     while task.active():
#       time.sleep(10)

#     self.ee_boundary_id = asset_id

#     return

#   def get_ee_boundary(self) -> ee.Geometry:
#     ee_asset = ee.FeatureCollection(self.ee_boundary_id)
#     geometry = ee_asset.geometry()

#     return geometry

In [ ]:
# SENTINEL-2 COLLECTION
sentinel_2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
salso_cascade_sentinel_2 = sentinel_2.filterBounds(salso_cascade_area)
salso_cascade_sentinel_2_2024 = salso_cascade_sentinel_2.filterDate('2024-01-01', '2024-12-31')
salso_cascade_sentinel_2_2024_cloud_free = salso_cascade_sentinel_2_2024.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))

# SENTINEL-2 RGB MOSAIC
salso_cascade_sentinel_2_rgb = salso_cascade_sentinel_2_2024_cloud_free.select(['TCI_R', 'TCI_G', 'TCI_B'])
salso_cascade_sentinel_2_rgb_mosaic = salso_cascade_sentinel_2_rgb.mosaic()
salso_cascade_sentinel_2_rgb_mosaic = salso_cascade_sentinel_2_rgb_mosaic.clip(salso_cascade_area)

In [ ]:
# SENTINEL-2 NDMI MOSAIC
salso_cascade_sentinel_2_2024_ndmi = salso_cascade_sentinel_2_2024_cloud_free.select(['B8', 'B11']).map(lambda image: image.normalizedDifference(['B8', 'B11']).rename('NDMI'))
salso_cascade_sentinel_2_2024_ndmi_mosaic = salso_cascade_sentinel_2_2024_ndmi.mosaic()
salso_cascade_sentinel_2_2024_ndmi_mosaic = salso_cascade_sentinel_2_2024_ndmi_mosaic.clip(salso_cascade_area)

# SENTINEL-2 NDWI MOSAIC
salso_cascade_sentinel_2_2024_ndwi = salso_cascade_sentinel_2_2024_cloud_free.select(['B3', 'B8']).map(lambda image: image.normalizedDifference(['B3', 'B8']).rename('NDWI'))
salso_cascade_sentinel_2_2024_ndwi_mosaic = salso_cascade_sentinel_2_2024_ndwi.mosaic()
salso_cascade_sentinel_2_2024_ndwi_mosaic = salso_cascade_sentinel_2_2024_ndwi_mosaic.clip(salso_cascade_area)

In [ ]:
# LANDSAT-8 COLLECTION
landsat_8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
salso_cascade_landsat_8 = landsat_8.filterBounds(salso_cascade_area)
salso_cascade_landsat_8_2024_cloud_free = salso_cascade_landsat_8.filter(ee.Filter.lt('CLOUD_COVER', 10))

# LANDSAT-8 SALSO CASCADE RGB MOSAIC
salso_cascade_landsat_8_rgb = salso_cascade_landsat_8_2024_cloud_free.select(['SR_B4', 'SR_B3', 'SR_B2'])
salso_cascade_landsat_8_rgb_mosaic = salso_cascade_landsat_8_rgb.mosaic()
salso_cascade_landsat_8_rgb_mosaic = salso_cascade_landsat_8_rgb_mosaic.clip(salso_cascade_area)

In [ ]:
# LANDSAT-8 SALSO CASCADE TIRS MOSAIC
salso_cascade_landsat_8_tirs = salso_cascade_landsat_8_2024_cloud_free.select(['ST_TRAD'])
salso_cascade_landsat_8_tirs_mosaic = salso_cascade_landsat_8_tirs.mosaic()
salso_cascade_landsat_8_tirs_mosaic = salso_cascade_landsat_8_tirs_mosaic.clip(salso_cascade_area).reproject('EPSG:4326', scale=1)

In [ ]:
# LANDSAT-8 SURFACE TEMPERATURE
constant_1 = ee.Image.constant(0.00341802)
constant_2 = ee.Image.constant(149)
constant_3 = ee.Image.constant(273.15)

salso_cascade_landsat_8_st_kelvin = ee.Image(salso_cascade_landsat_8_tirs_mosaic).multiply(constant_1).add(constant_2)

salso_cascade_landsat_8_st_celsius = salso_cascade_landsat_8_st_kelvin.subtract(constant_3)

In [ ]:
dem = DigitalElevationModel('NASA/NASADEM_HGT/001', band='elevation', bounds=salso_cascade_area)
salso_cascade_dem = dem.image
salso_cascade_dem_hillshade = dem.hillshade(-329.88, 40.78)
salso_cascade_dem_aspect = dem.aspect()

In [ ]:
params = {'region': salso_cascade_area,
          'scale': 10,
          'format': 'GEO_TIFF'}

dem_url = salso_cascade_dem.getDownloadURL(params)
# Writer.ee_image_to_colab(salso_cascade_dem, 'dem.tif', params)

In [ ]:
map = geemap.Map()
map.add_basemap('SATELLITE')
map.centerObject(salso_cascade_point, 15)

# DEM
# map.addLayer(salso_cascade_landsat_8_tirs_mosaic,
#              VisualizationParameters.min_and_max(salso_cascade_landsat_8_tirs_mosaic,
#                                                  'ST_TRAD',
#                                                  palette=Palettes.rainbow()),
#              name='Landsat 9 TIRS')

# map.addLayer(salso_cascade_landsat_8_tirs_mosaic,
#              VisualizationParameters.cumulative_cut(salso_cascade_landsat_8_tirs_mosaic,
#                                                     'ST_TRAD',
#                                                     palette=Palettes.rainbow(),
#                                                     min=2,
#                                                     max=97),
#              name='Landsat 9 TIRS - cumulative_cut')

map.addLayer(salso_cascade_landsat_8_st_celsius,
            #  VisualizationParameters.cumulative_cut(salso_cascade_landsat_8_st_celsius,
            #                                         'ST_TRAD',
            #                                         palette=Palettes.rainbow(),
            #                                         min=2,
            #                                         max=97),
             {'palette': Palettes.rainbow(), 'min': 10, 'max':33},
             name='Landsat 9 Celsius - cumulative_cut',
             shown=False)

map

Map(center=[-30.569416210967997, -53.448807548002016], controls=(WidgetControl(options=['position', 'transpare…